In [68]:
import os
path='/home/ubuntu/workspace/t1/'
pypath='/home/ubuntu/workspace/t1/PythonCode'
os.chdir(path)
import django
django.setup()
os.chdir(pypath)

from openpyxl import load_workbook, Workbook, cell
import pandas as pd, numpy as np
from datetime import datetime
from prof.ProcessingFunctions import ifHeaderReturnIndex, isRowValid, makeBold, fixWidth
from django.shortcuts import render
from django.contrib.auth.decorators import login_required
from django.http import HttpResponse
from prof.forms import FileSpaceForm
from prof.models import FileSpace, RunSpace, TransactionData, PnLData
from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
from guardian.shortcuts import get_objects_for_user, assign_perm, get_perms
from django.contrib.auth.models import User
from guardian.decorators import permission_required_or_403
from django.db import transaction
from prof.settings import Key_settings_dict, Transactions_settings_dict


def createKeyTemplate(transactionfile):
    Tr = pd.read_excel(transactionfile)
    Tr['Month']=Tr['Date'].apply(lambda x: str(x.year)+'-'+ ('0'+str(x.month))[-2:])
    KeyTemplate=Tr[['Business Unit','Product Number','Month', 'List Price']].groupby( ['Business Unit','Product Number','Month']).sum().round().unstack()
    KeyTemplate.columns=KeyTemplate.columns.droplevel(0)
    KeyTemplate.columns.name=None
    KeyTemplate[KeyTemplate!=np.nan]=None
    KeyTemplate=KeyTemplate.reset_index()
    KeyTemplate[lastColumnName] = np.nan
    return KeyTemplate
    
def createQuantityKey_old(transactionfile):
    Tr = pd.read_excel(transactionfile)
    Tr['Month']=Tr['Transaction Date'].apply(lambda x: str(x.year)+'-'+ ('0'+str(x.month))[-2:])
    QKey=Tr[['Business Unit','Product Number','Month', 'Quantity']].groupby( ['Business Unit','Product Number','Month']).sum().round().unstack()
    QKey.columns=QKey.columns.droplevel(0)
    QKey.columns.name=None
    #QKey[QKey!=np.nan]=None
    #QKey['Trailing Twelve Months'] = np.nan
    QKeyAgg=Tr[['Business Unit','Product Number', 'Quantity']].groupby( ['Business Unit','Product Number']).sum().round()
    QKeyAgg.rename(columns={'Quantity': Key_settings_dict['last_col_name']},inplace=True)
    QKey=pd.concat([QKey,QKeyAgg],axis=1)
    QKey=QKey.reset_index().fillna(0)
    return QKey

def createQuantityKey(run, Transactions_settings_dict= Transactions_settings_dict, Key_settings_dict=Key_settings_dict):
    wb=Workbook()
    wb.remove_sheet(wb.active)
    ws = wb.create_sheet('Quantity')
    Tr=pd.DataFrame.from_records(TransactionData.objects.filter(run=run).values())
    Tr['Month'] = Tr['TransactionDate'].apply(lambda x: datetime.strftime(x,'%b-%Y'))
    key_mont_column_names_sorted = sorted(set(Tr['Month']), key=lambda day: datetime.strptime(day, "%b-%Y"))
    QKey=Tr[['BusinessUnit','ProductNumber','Month', 'Quantity']].groupby( ['BusinessUnit','ProductNumber','Month']).sum().round().unstack()
    QKey.columns=QKey.columns.droplevel(0)
    QKey.columns.name=None
    #QKey[QKey!=np.nan]=None
    #QKey['Trailing Twelve Months'] = np.nan
    QKeyAgg=Tr[['BusinessUnit','ProductNumber', 'Quantity']].groupby( ['BusinessUnit','ProductNumber']).sum().round()
    QKeyAgg.rename(columns={'Quantity': Key_settings_dict['last_col_name']},inplace=True)
    QKey=pd.concat([QKey,QKeyAgg],axis=1)
    QKey=QKey.reset_index().fillna(0)
    QKey = QKey[['BusinessUnit','ProductNumber']+key_mont_column_names_sorted+[Key_settings_dict['last_col_name']]]
    row=[]
    for v in QKey.columns:
        newcell = cell.Cell(ws,column='A', row=1,value=v)
        newcell.font=Font(bold=True)
        row.append(newcell)
    ws.append(row)
    for index, row in QKey.iterrows():
        ws.append(list(row))
    row = ws.row_dimensions[1]
    row.font = Font(bold=True)
    fixWidth(ws)
    return(wb)

In [69]:
run=RunSpace.objects.get(pk=1)
transactionfile = 'SampleTransactionData.xlsx'

In [72]:
test='test.xlsx'
createQuantityKey(run).save(test)